In [55]:
import os
from getpass import getpass
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import faiss
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import LLMChain





os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")

OpenAI API key 입력:  ········


In [57]:
# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")



In [59]:
# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("/Users/t2023-m0072/Desktop/Git-class/인공지능산업최신동향_2024년11월호.pdf")

# 페이지 별 문서 로드
docs = loader.load()

In [67]:
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

char_splits = text_splitter.split_documents(docs)

print("CharacterTextSplitter 결과 (상위 10개):")
for i, chunk in enumerate(char_splits[:10]):
    print(f"청킹 {i+1}: {chunk.page_content}") # RecursiveCharacterTextSplitter 설정


CharacterTextSplitter 결과 (상위 10개):
청킹 1: 2024년 11월호
청킹 2: 2024년 11월호
Ⅰ. 인공지능 산업 동향 브리프 1. 정책/법제    ▹ 미국 민권위원회, 연방정부의 얼굴인식 기술 사용에 따른 민권 영향 분석························1   ▹ 미국 백악관 예산관리국, 정부의 책임 있는 AI 조달을 위한 지침 발표·····························2   ▹ 유로폴, 법 집행에서 AI의 이점과 과제를 다룬 보고서 발간··············································3   ▹ OECD, 공공 부문의 AI 도입을 위한 G7 툴킷 발표··························································4   ▹ 세계경제포럼, 생성AI 시대의 거버넌스 프레임워크 제시····················································5  2. 기업/산업    ▹ CB인사이츠 분석 결과, 2024년 3분기 벤처 투자 31%가 AI 스타트업에 집중··············6   ▹ 메타, 동영상 생성AI 도구 ‘메타 무비 젠’ 공개···································································7   ▹ 메타, 이미지와 텍스트 처리하는 첫 멀티모달 AI 모델 ‘라마 3.2’ 공개···························8   ▹ 앨런AI연구소, 벤치마크 평가에서 GPT-4o 능가하는 성능의 오픈소스 LLM ‘몰모’ 공개····9   ▹ 미스트랄AI, 온디바이스용 AI 모델 ‘레 미니스트로’ 공개·················································10   ▹ 카카오, 통합 AI 브랜드 겸 신규 AI 서비스 ‘카나나’ 공개···············································11 

In [65]:
recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,        
    chunk_overlap=10,      
    length_function=len,   
    is_separator_regex=False
)

# 청킹 수행
recursive_splits = recursive_text_splitter.split_documents(docs)

# 상위 10개 청킹된 내용 출력
print("\nRecursiveCharacterTextSplitter 결과 (상위 10개):")
for i, chunk in enumerate(recursive_splits[:10]):
    print(f"청킹 {i+1}: {chunk.page_content}") 


RecursiveCharacterTextSplitter 결과 (상위 10개):
청킹 1: 2024년 11월호
청킹 2: 2024년 11월호
청킹 3: Ⅰ. 인공지능 산업 동향 브리프 1. 정책/법제    ▹ 미국 민권위원회, 연방정부의 얼굴인식 기술 사용에 따른 민권 영향 분석························1
청킹 4: ▹ 미국 백악관 예산관리국, 정부의 책임 있는 AI 조달을 위한 지침 발표·····························2   ▹ 유로폴, 법 집행에서 AI의 이점과
청킹 5: AI의 이점과 과제를 다룬 보고서 발간··············································3   ▹ OECD, 공공 부문의 AI 도입을 위한 G7
청킹 6: 도입을 위한 G7 툴킷 발표··························································4   ▹ 세계경제포럼, 생성AI 시대의
청킹 7: 생성AI 시대의 거버넌스 프레임워크 제시····················································5  2. 기업/산업    ▹ CB인사이츠
청킹 8: ▹ CB인사이츠 분석 결과, 2024년 3분기 벤처 투자 31%가 AI 스타트업에 집중··············6   ▹ 메타, 동영상 생성AI 도구 ‘메타 무비 젠’
청킹 9: ‘메타 무비 젠’ 공개···································································7   ▹ 메타, 이미지와 텍스트
청킹 10: 이미지와 텍스트 처리하는 첫 멀티모달 AI 모델 ‘라마 3.2’ 공개···························8   ▹ 앨런AI연구소, 벤치마크 평가에서 GPT-4o


## 1. CharacterTextSplitter
- **기능**: 텍스트를 특정 `separator`를 기준으로 분리.
- **주요 파라미터**:
  - `separator`: 텍스트를 분리할 기준 문자열. (예: `"\n\n"`)
  - `chunk_size`: 각 청킹의 최대 문자 길이. (기본값: 100)
  - `chunk_overlap`: 청킹 간 중첩 길이. (기본값: 10)
  - `length_function`: 텍스트 길이를 계산하는 함수. (기본값: `len`)
  - `is_separator_regex`: `separator`가 정규표현식인지 여부. (기본값: `False`)
- **특징**:
  - 단순한 구조로 고정된 기준으로 청킹하기 적합.
  - 구분자가 문서에 없을 경우 분리가 제대로 이루어지지 않을 수 있음.

## 2. RecursiveCharacterTextSplitter
- **기능**: 문장을 더 작은 단위로 반복적으로 나누어 분리.
- **주요 파라미터**:
  - `chunk_size`: 각 청킹의 최대 문자 길이. (기본값: 100)
  - `chunk_overlap`: 청킹 간 중첩 길이. (기본값: 10)
  - `length_function`: 텍스트 길이를 계산하는 함수. (기본값: `len`)
  - `is_separator_regex`: `separator`가 정규표현식인지 여부. (기본값: `False`)
- **특징**:
  - `CharacterTextSplitter`보다 유연하게 작동하며, 청킹 실패 가능성이 적음.
  - 더 복잡한 문서 구조에서 효과적으로 동작.

## 3. 차이점 요약
| 파라미터        | CharacterTextSplitter                                  | RecursiveCharacterTextSplitter                   |
|----------------|------------------------------------------------------|------------------------------------------------|
| `separator`    | 기준 문자열을 지정해 분리.                              | 더 작은 단위로 반복적으로 나눔.                   |
| `chunk_size`   | 지정된 크기 내에서 분리.                                | 지정된 크기 내에서 더 유연하게 처리.              |
| 장점           | 단순하고 빠르게 분리.                                   | 복잡한 구조에서도 청킹 실패 가능성이 적음.         |
| 단점           | 구분자가 없거나 부적절할 경우 의도치 않은 결과를 생성.      | 처리 시간이 약간 더 오래 걸릴 수 있음.            |


In [71]:
# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


In [79]:
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

In [87]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [89]:
# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

In [91]:

class SimplePassThrough:
    def invoke(self, inputs, **kwargs):
        return inputs

class ContextToPrompt:
    def __init__(self, prompt_template):
        self.prompt_template = prompt_template
    
    def invoke(self, inputs):
        # 문서 내용을 텍스트로 변환
        if isinstance(inputs, list):
            context_text = "\n".join([doc.page_content for doc in inputs])
        else:
            context_text = inputs
        
        # 프롬프트 템플릿에 적용
        formatted_prompt = self.prompt_template.format_messages(
            context=context_text,
            question=inputs.get("question", "")
        )
        return formatted_prompt

# Retriever를 invoke() 메서드로 래핑하는 클래스 정의
class RetrieverWrapper:
    def __init__(self, retriever):
        self.retriever = retriever

    def invoke(self, inputs):
        if isinstance(inputs, dict):
            query = inputs.get("question", "")
        else:
            query = inputs
        # 검색 수행
        response_docs = self.retriever.get_relevant_documents(query)
        return response_docs

llm_chain = LLMChain(llm=model, prompt=contextual_prompt)

# RAG 체인 설정
rag_chain_debug = {
    "context": RetrieverWrapper(retriever),
    "prompt": ContextToPrompt(contextual_prompt),
    "llm": model
}

/var/folders/vz/672zdnsn4bs4gb0mnjthpsyc0000gn/T/ipykernel_74243/2412360451.py:39: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  llm_chain = LLMChain(llm=model, prompt=contextual_prompt)


In [ ]:

# 챗봇 구동
while True:
    print("========================")
    query = input("질문을 입력하세요 : ")
    
    # 1. Retriever로 관련 문서 검색
    response_docs = rag_chain_debug["context"].invoke({"question": query})
    
    # 2. 문서를 프롬프트로 변환
    prompt_messages = rag_chain_debug["prompt"].invoke({
        "context": response_docs,
        "question": query
    })
    
    # 3. LLM으로 응답 생성
    response = rag_chain_debug["llm"].invoke(prompt_messages)
    
    print("\n답변:")
    print(response.content)

질문을 입력하세요 :  최신 인공지능 산업동향에 대해 알려줘


/var/folders/vz/672zdnsn4bs4gb0mnjthpsyc0000gn/T/ipykernel_74243/2412360451.py:36: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  response_docs = self.retriever.get_relevant_documents(query)



답변:
최신 인공지능 산업동향에 따르면, 2024년 11월호에서는 다음과 같은 주요 내용이 다루어졌습니다:

1. **정책/법제**:
   - 미국 민권위원회가 얼굴인식 기술의 민권 영향 분석을 발표했습니다.
   - 미국 백악관 예산관리국이 정부의 책임 있는 AI 조달을 위한 지침을 발표했습니다.
   - 유로폴이 AI의 법 집행에서의 이점과 과제를 다룬 보고서를 발간했습니다.
   - OECD가 공공 부문의 AI 도입을 위한 G7 툴킷을 발표했습니다.
   - 세계경제포럼이 생성AI 시대의 거버넌스 프레임워크를 제시했습니다.

2. **기업/산업**:
   - CB인사이츠의 분석에 따르면, 2024년 3분기 벤처 투자 중 31%가 AI 스타트업에 집중되었습니다.
   - 메타는 동영상 생성 AI 도구인 ‘메타 무비 젠’을 공개했습니다.
   - 메타는 이미지와 텍스트를 처리하는 첫 멀티모달 AI 모델 ‘라마 3.2’를 발표했습니다.
   - 앨런AI연구소는 GPT-4o를 능가하는 성능의 오픈소스 LLM ‘몰모’를 공개했습니다.
   - 미스트랄AI는 온디바이스용 AI 모델 ‘레 미니스트로’를 발표했습니다.
   - 카카오는 통합 AI 브랜드 ‘카나나’를 공개했습니다.

3. **기술/연구**:
   - 2024년 노벨 물리학상과 화학상은 AI 관련 연구자들이 수상했습니다.
   - 미국 국무부는 AI 연구에서 국제협력을 위한 ‘글로벌 AI 연구 의제’를 발표했습니다.
   - 일본 AI안전연구소는 AI 안전성 평가 가이드를 발간했습니다.
   - 구글 딥마인드는 반도체 칩 레이아웃 설계하는 AI 모델 ‘알파칩’을 발표했습니다.
   - AI21 CEO는 AI 에이전트에 트랜스포머 아키텍처의 대안 필요성을 강조했습니다.

4. **인력/교육**:
   - MIT 산업성과센터는 자동화 기술의 근로자 영향 조사를 실시했습니다.
   - 다이스 조사에 따르면 AI 전문가의 73%가 2025년 중 이직을 고려하고 있습니다.
   - 가트너는 AI로 인해 엔지니어링

# RAG (Retrieval-Augmented Generation)의 필요성

## 1. LLM의 한계 극복
- **최신 정보 부족**: LLM은 학습된 시점 이후의 정보를 반영하지 못함.
- **정보의 신뢰성**: 잘못된 정보 생성 가능성.
- **훈련 데이터 제한**: 특정 도메인에 대한 세부 정보 부족.

## 2. 대규모 데이터에 대한 실시간 접근
- **정확한 답변 제공**: 외부 데이터베이스나 검색을 통해 최신 정보 제공.
- **도메인 지식 강화**: 특정 분야(법률, 의학 등) 질문에 더 정확한 답변 가능.

## 3. 데이터 보강을 통한 성능 향상
- **훈련 비용 절감**: 모든 데이터를 훈련시키지 않고 필요한 데이터만 검색하여 처리.
- **문맥 보강**: 외부 데이터를 통해 답변의 품질과 정확성 향상.

## 4. 유연한 데이터 처리
- **대용량 데이터 사용 가능**: 외부 데이터베이스 활용으로 입력 크기 제한 없음.
- **사용자 정의 가능**: 특정 검색 알고리즘이나 데이터베이스 설정 가능.
